In [2]:
#importing required libraries
import mediapipe as mp 
import cv2 

import csv
import os
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [3]:
from collections import Counter

In [4]:
from flask import Flask
from flask_restful import Resource, Api
from flask_cors import CORS

In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
import datetime

In [6]:
app = Flask("VideoAPI")
cors = CORS(app)
cors = CORS(app, origins=['http://localhost:3000'])
api = Api(app)


# Initialize Firebase Admin SDK
# cred = credentials.Certificate("i-binder-353018-firebase-adminsdk-57u1c-95284f8bdc.json")
# storageapp = firebase_admin.initialize_app(cred, {
#     'storageBucket': 'i-binder-353018.appspot.com'
# },"u10")
# firebase_admin.initialize_app(cred,{},"u6")

# Initialize Firebase app
cred = credentials.Certificate('i-binder-353018-firebase-adminsdk-57u1c-95284f8bdc.json')
firebase_admin.initialize_app(cred, {'storageBucket':'i-binder-353018.appspot.com'})

# Check if credentials are valid
# try:
#     firebase_admin.get_app()
#     print("Credentials are valid!")
# except ValueError as e:
#     print("Invalid credentials:", e)

In [7]:
# Autistic Behaviour Classification Class
class AutismBehaviourClassifier(Resource):
    
    def get(self):
        
        

        # Get video file from Firebase Storage
#         bucket = storage.bucket()
#         blob = bucket.blob('/Video/123.mp4')
#         url = blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET')

        # Download the video from Firebase Storage
        bucket = storage.bucket()
        
        blob = bucket.blob('Video/123.mp4')
        video_file = '123.mp4'
        blob.download_to_filename(video_file)
        
        classification_results = []
        
        #mediapipe holistic solution
        mp_drawing = mp.solutions.drawing_utils
        mp_holistic = mp.solutions.holistic
        
        with open('body_language.pkl', 'rb') as f:
            model = pickle.load(f)
            
        cap = cv2.VideoCapture(video_file)
        # Initiate holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.75, min_tracking_confidence=0.75) as holistic:
    
            while cap.isOpened():
                ret, frame = cap.read()
        
                # Check if the frame was successfully read
                if not ret:
                # If not, the end of the video has been reached
                    break
        
                # Recolor Feed
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        
        
                # Make Detections
                results = holistic.process(image)
                # print(results.face_landmarks)
        
                # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                # 1. Draw face landmarks
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                         )
        
                # 2. Right hand
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                         )

                # 3. Left Hand
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                         )

                # 4. Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                         )
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                    # Extract Face landmarks
                    face = results.face_landmarks.landmark
                    face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

                    # Concate rows
                    row = pose_row+face_row

                    # Make Detections
                    X = pd.DataFrame([row])
                    body_language_class = model.predict(X)[0]
                    body_language_prob = model.predict_proba(X)[0]
                    body_language_prob_result = round(body_language_prob[np.argmax(body_language_prob)],2)
                    
                    if body_language_prob_result > 0.9:
                        print(body_language_class.split(' ')[0], body_language_prob_result)
                        classification_results.append(body_language_class.split(' ')[0])

                        # Grab ear coords
                        coords = tuple(np.multiply(
                                        np.array(
                                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                             results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                                    , [640,480]).astype(int))

                        cv2.rectangle(image, 
                                      (coords[0], coords[1]+5), 
                                      (coords[0]+len(body_language_class)*20, coords[1]-30), 
                                      (245, 117, 16), -1)
                        cv2.putText(image, body_language_class, coords, 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                        # Get status box
                        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                        # Display Class
                        cv2.putText(image, 'CLASS'
                                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 1, cv2.LINE_AA)
                        cv2.putText(image, body_language_class.split(' ')[0]
                                    , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                        # Display Probability
                        cv2.putText(image, 'PROB'
                                    , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 1, cv2.LINE_AA)
                        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                                    , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                except:
                    pass
        
        # Create a counter object from the list of classification results
        counts = Counter(classification_results)
        # Print the counts of each category
#         print(counts)

        # Decide on the overall classification based on the category with the highest count
        if counts['Hand_Flickering'] > counts['Head_Banging']:
            overall_classification = 'Hand_Flickering'
        else:
            overall_classification = 'Head Banging'

        return "Overall classification : " + overall_classification



In [ ]:
api.add_resource(AutismBehaviourClassifier, '/VideoClassify')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "VideoAPI" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 

Head_Banging 0.99


C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Head_Banging 1.0
Head_Banging 1.0


...

C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Head_Banging 1.0


C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Geethan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [26/Apr/2023 21:48:07] "GET /VideoClassify HTTP/1.1" 200 -


Head_Banging 1.0


127.0.0.1 - - [26/Apr/2023 21:48:08] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# This should be added to the requirement.txt when deploying 
# mediapipe==0.9.0.1
# cv2==4.6.0
# csv==1.0
# np==1.21.5
# pd==1.4.2